In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import QuantumCircuit
from qiskit.providers.basic_provider import BasicSimulator
from qiskit import transpile
import numpy as np

In [ ]:
# 1.2 QFT acting on first n b its
def apply_qft(n,circuit):
    circuit.barrier()

    # Hadamard gate outer loop
    for j in range(n-1, -1, -1):
        
        circuit.h(j)
        
        for k in range(j-1, -1, -1):
            theta = np.pi / 2**(j-k)
            circuit.cp(theta, j, k)

        circuit.barrier()
    
    # Swap qubit i with qubit n-i-1
    for i in range(n // 2):
        circuit.swap(i, n-i-1)

    return circuit

In [17]:
# 1.2 check
n = 3
# n = 4
circuit = QuantumCircuit(n,0)
apply_qft(n,circuit)
print(circuit)
print()

      ░                         ░                ░ ┌───┐ ░    
q_0: ─░────────────────■────────░───────■────────░─┤ H ├─░──X─
      ░                │        ░ ┌───┐ │P(π/2)  ░ └───┘ ░  │ 
q_1: ─░───────■────────┼────────░─┤ H ├─■────────░───────░──┼─
      ░ ┌───┐ │P(π/2)  │P(π/4)  ░ └───┘          ░       ░  │ 
q_2: ─░─┤ H ├─■────────■────────░────────────────░───────░──X─
      ░ └───┘                   ░                ░       ░    



In [30]:
# 1.3 Inverse QFT
def apply_iqft(n, circuit):

    # Starting with end of `apply_qft()`
    # Swap qubit i with qubit n-i-1
    for i in range(n // 2):
        circuit.swap(i, n-i-1)

    circuit.barrier()

    # Loop through in opposite order
    for j in range(0, n):
        
        # Original k and j targets flipped & theta negated
        for k in range(0,j):
            theta = np.pi / 2**(j-k)
            circuit.cp(-theta, k, j)

        # Inverse of Hadamard is Hadamard
        circuit.h(j)
        circuit.barrier()
    

    return circuit

In [31]:
# 1.3 check
n = 4
circuit = QuantumCircuit(n,0)
apply_iqft(n, circuit)
print(circuit)
print()

            ░ ┌───┐ ░                 ░                           ░           »
q_0: ─X─────░─┤ H ├─░──■──────────────░──■────────────────────────░──■────────»
      │     ░ └───┘ ░  │P(-π/2) ┌───┐ ░  │                        ░  │        »
q_1: ─┼──X──░───────░──■────────┤ H ├─░──┼─────────■──────────────░──┼────────»
      │  │  ░       ░           └───┘ ░  │P(-π/4)  │P(-π/2) ┌───┐ ░  │        »
q_2: ─┼──X──░───────░─────────────────░──■─────────■────────┤ H ├─░──┼────────»
      │     ░       ░                 ░                     └───┘ ░  │P(-π/8) »
q_3: ─X─────░───────░─────────────────░───────────────────────────░──■────────»
            ░       ░                 ░                           ░           »
«                               ░ 
«q_0: ──────────────────────────░─
«                               ░ 
«q_1: ─■────────────────────────░─
«      │                        ░ 
«q_2: ─┼─────────■──────────────░─
«      │P(-π/4)  │P(-π/2) ┌───┐ ░ 
«q_3: ─■─────────■────────┤ H ├─░─


In [ ]:
# 1.4 Generalizing the functions
def apply_qft_gen(qubits, circuit):
    circuit.barrier()

    q_len = len(qubits)

    for j in range(q_len -1, -1, -1):
        
        target = qubits[j]
        circuit.h(target)
        
        for k in range(j-1, -1, -1):
            control = qubits[k]
            theta = np.pi / 2**(j-k)
            circuit.cp(theta, control, target)

        circuit.barrier()
    
    for i in range(q_len // 2):
        circuit.swap(qubits[i], qubits[q_len-i-1])
    
    return circuit

def apply_iqft_gen(qubits, circuit):

    q_len = len(qubits)

    # Starting with end of `apply_qft()`
    # Swap qubit i with qubit n-i-1
    for i in range(q_len // 2):
        circuit.swap(qubits[i], qubits[q_len-i-1])

    circuit.barrier()

    # Loop through in opposite order
    for j in range(0, q_len):
        
        target = qubits[j]

        # Original k and j targets flipped & theta negated
        for k in range(0,j):
            control = qubits[k]
            theta = np.pi / 2**(j-k)
            circuit.cp(-theta, control, target)

        # Inverse of Hadamard is Hadamard
        circuit.h(target)
        circuit.barrier()

    return circuit